#### Prerequisites

- Basic familiarity with [Numpy](https://numpy.org/doc/stable/user/quickstart.html)
- Basic familiarity with [Pyplot](https://matplotlib.org/stable/tutorials/introductory/pyplot.html)

<a name="outline"></a>

## Outline

- [Section 1](#section-1): Intro to kNN
- [Section 2](#section-2): Classification with kNN
- [Section 3](#section-3): Hyperparameter Tuning for $k$ with _T-fold_ Cross Validation
- [Section 4](#section-4): Regression with kNN


# _k_ nearest neighbours (kNN)

The purpose of this notebook is to understand and implement the kNN algorithm. You are not allowed to use any package that has a complete kNN framework implemented (e.g., scikit-learn).

<a name="section-1"></a>

## Section 1: Intro to kNN [^](#outline)

The kNN algorithm can be used both for classification and  regression. Broadly speaking, it start with calculating the distance of a given point $x$ to all other points in the data set. Then, it finds the _k_ nearest points closest to $x$, and assigns the new point $x$ to the majority class of the _k_ nearest points _(classification)_. So, for example, if two of the _k_=3 closest points to $x$ were red while one is blue, $x$ would be classified as red.

On the other hand in _regression_, we see the labels as continuous variables and assign the label of a data point $x$ as the mean of the labels of its _k_ nearest neighbours. 

In [ ]:
import numpy as np
from sklearn.datasets import make_classification, fetch_california_housing
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

Important things first: You already know that the kNN algorithm is based on computing distances between data points. So, let's start with defining a function that computes such a distance. For simplicity, we will only work with **Euclidean distances** in this notebook, but other distances can be chosen interchangably, of course.

Implement in the following cell the Euclidean distance $d$, defined as
$$
d(\boldsymbol p, \boldsymbol q) = \sqrt{\sum_{i=1}^D{(q_i-p_i)^2}} \, ,
$$
where $\boldsymbol p$ and $\boldsymbol q$ are the two points in our $D$-dimensional Euclidean space.


In [ ]:
## EDIT THIS FUNCTION
def euclidian_distance(p, q):
    return ... ## <-- EDIT THIS LINE

<a name="section-2"></a>

## Section 2: Classification with kNN [^](#outline)

We start with using kNN for classification tasks, and create a dataset with sklearn's [`make_classification`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html) function and standardise the data.

In [ ]:
X_class, y = make_classification(n_samples=100, n_features=2, n_redundant=0, n_informative=2, n_clusters_per_class=1, n_classes=3, random_state=15)

In [ ]:
## EDIT THIS FUNCTION
def standardise(X):
  mu = np.mean(X, 0)
  sigma = np.std(X, 0)
  X_std = ... ## <-- EDIT THIS LINE
  return X_std

In [ ]:
X = standardise(X_class)

As with any other supervised machine learning method, we create a training and test set to learn and evaluate our model, respectively.

In [ ]:
# shuffling the rows in X and y
p = np.random.permutation(len(y))
X = X[p]
y = y[p]

# we split train to test as 70:30
split_rate = 0.7
X_train, X_test = np.split(X, [int(split_rate*(X.shape[0]))])
y_train, y_test = np.split(y, [int(split_rate*(y.shape[0]))])

We visualise the data set with points in the training set being fully coloured and points in the test being half-transparent.

In [ ]:
# define colormaps
cm = plt.cm.RdBu
cm_bright = ListedColormap(['blue', 'red', 'green'])

# visual exploration
plt.figure(figsize=(12,8))
plt.xlabel(r'$X^{(1)}$', size=24)
plt.ylabel(r'$X^{(2)}$', size=24)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright)
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.25)
plt.show()

We try to find the _k_ nearest neighbours in our training set for every test data point. The majority of labels of the _k_ closest training points determines the label of the test point. 

In [ ]:
## EDIT THIS FUNCTION
def k_neighbours(X_train, X_test, k=5, return_distance=False):
  n_neighbours = k
  dist = []
  neigh_ind = []
  
  # compute distance from each point x_text in X_test to all points in X_train (hint: use python's list comprehension: [... for x_test in X_test])
  point_dist = ... ## <-- EDIT THIS LINE

  # determine which k training points are closest to each test point
  for row in point_dist:
      enum_neigh = enumerate(row)
      sorted_neigh = sorted(enum_neigh, key=lambda x: x[1])[:k]

      ind_list = [tup[0] for tup in sorted_neigh]
      dist_list = [tup[1] for tup in sorted_neigh]

      dist.append(dist_list)
      neigh_ind.append(ind_list)
  
  # return distances together with indices of k nearest neighbouts
  if return_distance:
      return np.array(dist), np.array(neigh_ind)
  
  return np.array(neigh_ind)

Once we know which _k_ neighbours are closest to our test points, we can predict the labels of these test points.

We implement this in a "pythonic" way and call the previous function `k_neighbours` _within_ the next function `predict`.

Our `predict` function determines how any point $x_\text{test}$ in the test set is classified. Here, we only consider the case where each of the *k* neighbours contributes equally to the classification of $x_\text{test}$.

In [ ]:
## EDIT THIS FUNCTION
def predict(X_train, y_train, X_test, k=5):
  # each of the k neighbours contributes equally to the classification of any data point in X_test  
  neighbours = k_neighbours(X_test, k=k)
  # count number of occurences of label with np.bincount and choose the label that has most with np.argmax (hint: use python's list comprehension: np.array([... for neighbour in neighbours]))
  y_pred = ... ## <-- EDIT THIS LINE

  return y_pred

To evaluate the algorithm in a more principled way, we need to implement a function that computes the mean accuracy by counting how many of the test points have been classified correctly and dividing this number by the total number of data points in our test set.

Again, we do this is in a pythonic way and call the previous `predict` function _within_ the next function `score`.

In [ ]:
## EDIT THIS FUNCTION
def score(X_train, y_train, X_test, y_test, k=5):
  y_pred = ... ## <-- EDIT THIS LINE
  return np.mean(y_pred==y_test)

It is quite common to print both the training and test set accuracies.

In [ ]:
k = 8
print('Training set mean accuracy:', score(X_train, y_train, X_train, y_train, k=k))
print('Test set mean accuracy:', score(X_train, y_train, X_test, y_test, k=k))

#### Questions
1. Does the solution above look reasonable?
2. Play around with different values for _k_. How does it influence the classification mean accuracy?
3. Compare the training and test set accuracy. Is there a difference? If so, what does the difference tell you?
4. Choose different ratios for the split between training and test set, and re-run the entire algorithm. What can you learn from different ratios?
5. Considering an accuracy estimate on a test-split wich contains 30% of the dataset examples is 0.86, do we guarantee to obtain the same accuracy estimate when we apply our model on infinetely large unseen test examples, i.e. does the accuracy of your model on the test-split generalize well on unseen data? From this week's lecture notes, what would you suggest to improve our confidence in the accuracy estimate, so it is more closer estimate to the true accuracy when testing on unseen examples? 

<a name="section-3"></a>

### 3 Hyperparameter Tuning for $k$ with _T-fold_ Cross [^](#outline)

Let's consider a systematic way to help select the best $k$ (i.e. the hyperparameter of $k$-NN). In previous cells, we splitted our data into 70%:30% for training:test examples. Now we need to choose the best $k$, but without looking at the accuracy on the test set (which should be used at the end to assess the predictive power of the model with the chosen $k$). To this end, we perform $T$-fold cross validation, where, **importantly**, we don't evaluate the accuracy of the model using the same examples on which we trained our model, rather on a held out validation set. We can achieve this by running $T$ experiments, and in each one we use disjoint partitions for the training and accuracy examples. By averaging the accuracy estimates over the $T$ experiments, we get a more precise and reliable accuracy estimate than before. If we consider $T=3$ folds, then we can run the three experiments and evaluate the average accuracy as in the figure below:

![cv](https://drive.google.com/uc?export=view&id=1aTZu9xNdgPb7_nBzs1cHrEpRjF31vK2M)


Finaly, we need to isolate a separate set, before using cross-validation for hyperparameter tuning, to test our model after selecting the best performing $k$ for $k$-NN, so we further consider the following partitioning, which we will need to adhere with throughout the future notebooks and courseworks (i.e. in the problems when we need to use cross-validation for hyperparameter tuning then test on completely unseen data that are not involved in hyperparamter tuning):

![cv2](https://drive.google.com/uc?export=view&id=1d2fIBBhT0lzcUCYSfKcvCEaJT2df9-9K)



Let's consider a ratio 80:20 for training and test splits

In [ ]:
# shuffling the rows in X and y
p = np.random.permutation(len(y))
X = X[p]
y = y[p]


# we split train to test as 80:20
split_rate = 0.8
X_train, X_test = np.split(X, [int(split_rate*(X.shape[0]))])
y_train, y_test = np.split(y, [int(split_rate*(y.shape[0]))])

Now let's partition our training split into 5-folds. We could store the corresponding indices only:

In [ ]:
# Now we have a list of five index arrays, each correspond to one of the five folds.
folds_indexes = np.split(np.arange(len(y_train)), 5)
folds_indexes

In [ ]:
# EDIT THIS FUNCTION
def cross_validation_score(X_train, y_train, folds, k):
  scores = []
  for i in range(len(folds)):
    val_indexes = folds[i]
    train_indexes = list(set(range(y_train.shape[0])) - set(val_indexes))
    
    X_train_i = X_train[train_indexes, :]
    y_train_i = y_train[train_indexes]


    X_val_i = X_train[42, :] # <- EDIT THIS LINE
    y_val_i = y_train[13] # <- EDIT THIS LINE

    score_i = score(edit, this, function, arguments, k=k) # <- EDIT THIS LINE
    scores.append(score_i)

  # Return the average score
  return max(scores) # <- EDIT THIS LINE

Let's scan a range of $k$ in $[1, 30]$ and select the one with the best cross-validation accuracy.

In [ ]:
def choose_best_k(X_train, y_train, folds, k_range):
  k_scores = np.zeros((len(k_range),))
  
  for i, k in enumerate(k_range):
    k_scores[i] = cross_validation_score(X_train, y_train, folds, k)
    print(f'CV_ACC@k={k}: {k_scores[i]:.3f}')

  best_k_index = np.argmax(k_scores)
  return k_range[best_k_index]


In [ ]:
best_k = choose_best_k(X_train, y_train, folds_indexes, np.arange(1, 31))

print('best_k:', best_k)


<a name="section-4"></a>

## Section 4:  Regression with kNN [^](#outline)

The kNN algorithm is mostly used for classification, but we can also utilise it for (non-linear) regression. Here, we calculate the label of every point in the test set as the mean of the _k_ nearest neighbours.

We start with defining a training set with sklearn's Californa housing data set. Note that this data set has normally 8 features, but we only extract the first feature, which corresponds to the median income in the district. The label is the median house value in the district.


In [ ]:
data = fetch_california_housing(return_X_y=True)
X = data[0][:,0].reshape((-1, 1))
y = data[1]

X_std = standardise(X)

As before, we first divide the data into training and test set:

In [ ]:
# shuffling the rows in X and y
p = np.random.permutation(len(y))
X = X_std[p]
y = y[p]

# we split train to test as 70:30
split_rate = 0.7
X_train, X_test = np.split(X, [int(split_rate*(X.shape[0]))])
y_train, y_test = np.split(y, [int(split_rate*(y.shape[0]))])

Let's plot it to get a sense how we can proceed. This time, we plot training examples in blue and test examples in red.

In [ ]:
# visual exploration
plt.figure(figsize=(12,8))
plt.xlabel(r"median income in '000 USD", size=20)
plt.ylabel(r"median house value in '00.000 USD", size=20)
plt.scatter(X_train, y_train, c='blue', alpha=0.25)
plt.scatter(X_test, y_test, c='red', alpha=0.25)
plt.show()

As before, we need to define a predicting function which we call `reg_predict`.

In [ ]:
## EDIT THIS FUNCTION
def reg_predict(X_train, y_train, X_test, k=20):
  # each of the k neighbours contributes equally to the classification of any data point in X_test  
  neighbours = k_neighbours(X_train, X_test, k=k)
  # compute mean over neighbours labels (hint: use python's list comprehension: np.array([... for neighbour in neighbours]))
  y_pred = ... ## <-- EDIT THIS LINE

  return y_pred

In [ ]:
# computing predictions... (takes a few minutes due to the high sample size)
k = 20
y_pred = reg_predict(X_train, y_train, X_test, k=k)

In [ ]:
# ... and plotting them
plt.figure(figsize=(12,8))
plt.xlabel(r"median income (standardised)", size=20)
plt.ylabel(r"median house value in '00.000 USD", size=20)
#plt.scatter(X_train, y_train, c='blue', alpha=0.25)
plt.scatter(X_test, y_test, c='red', alpha=0.25)
plt.scatter(X_test, y_pred, c='yellow', alpha=0.25)
plt.show()

To determine how well the prediction was, let us determine the $R^2$ score. The labels of the test set will be called $y$ and the predictions on the test data $\hat{y}$.
$$
R^2(y, \hat{y}) = 1 - \frac{\sum_{i=1}^n (y_i - \hat{y}_i)^2}{\sum_{i=1}^n (y_i - \bar{y})^2} \, ,
$$
where $\bar{y} = \frac{1}{n} \sum_{i=1}^n y_i$.

In [ ]:
## EDIT THIS FUNCTION
def r2_score(y_test, y_pred):
  numerator = ... ## <-- EDIT THIS LINE
  y_avg = ... ## <-- EDIT THIS LINE
  denominator = ... ## <-- EDIT THIS LINE
  return 1 - numerator/denominator

In [ ]:
print(r'R2 score:', r2_score(y_test, y_pred))

#### Questions
1. Does the solution above look reasonable? What does your $R^2$ value tell you?
2. Play around with different values for _k_. How does it influence the regression?
3. Like we did in classification, excercise with implementing cross-validation to find the best performing k on the regression task.
4. Compare the training and test set accuracy. Is there a difference? If so, what does the difference tell you?
5. Choose different ratios for the split between training and test set, and re-run the entire algorithm. What can you learn from different ratios?
6. Can you replicate your results using [sklearn](https://scikit-learn.org/stable/modules/neighbors.html)?
7. Based on sklearn's documentation, can you see any differences in the algorithms that are implemented in sklearn?